In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()


from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer("Linus4Lyf/test-food")


In [2]:
df = pd.read_csv("Nutrient_Vals/nut_val_df.csv")
sampled_df = df.sample(frac =.05)
len(sampled_df)

281

In [3]:
labels = [                
                "Fruits and Vegetables",              
                "Meat",
                "Dairy",                                                
            ]

In [4]:
import json
import requests


def generate_label(ex_1, labels):
    try:
        #Encode query and documents
        query = ex_1
        docs = labels
        query_emb = model.encode(query)
        doc_emb = model.encode(docs)

        #Compute dot score between query and all document embeddings
        scores = util.dot_score(query_emb, doc_emb)[0].cpu().tolist()

        #Combine docs & scores
        doc_score_pairs = list(zip(docs, scores))

        #Sort by decreasing score
        doc_score_pairs = sorted(doc_score_pairs, key=lambda x: x[1], reverse=True)        
        if doc_score_pairs[0][1] > 4.0:
            return doc_score_pairs[0][0]
        else:
            return ex_1.split(",")[0]
    except Exception as e:
        print(e)
        return " "

sampled_df["label_foods"] = sampled_df["Main food description"].progress_apply(lambda x: generate_label(x, labels))

100%|████████████████████████████████████████████████████████████████████████████████| 281/281 [00:36<00:00,  7.78it/s]
